In [1]:
from pathlib import Path
import datetime as dt

import pandas as pd

from pybenchmark.client import BenchmarkClient

In [2]:
URL = "https://benchmark-container-app.braveflower-04bf6ce2.uksouth.azurecontainerapps.io/"
# URL = "http://127.0.0.1:8000"
DATA_DIR = Path("../resources")

In [3]:
client = BenchmarkClient(url=URL)

# Users

In [4]:
user = {"name": "Jordan Begg", "email": "jordansbegg@outlook.com", "password": "BenchmarkPassword"}

In [5]:
try:
    response = client.users.create_user(**user)
    print(f"Created User {user['name']}")
except Exception as e:
    print(e)

Created User Jordan Begg


# Weights

In [6]:
WEEKDAYS = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3,
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6
    }

In [7]:
df = pd.read_excel(DATA_DIR / "weights.xlsx", sheet_name="Sheet2")
df = df.melt(id_vars="Date", value_vars=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
df["dow"] = df["variable"].map(WEEKDAYS)
df["Real Date"] = df.apply(axis=1, func = lambda x: dt.timedelta(days=x.dow) + x.Date)
df = df.drop(columns=["Date", "dow", "variable"]).rename(columns={"value": "weight", "Real Date": "date"}).dropna()
weights = df.to_dict(orient="records")
weights

[{'weight': 60.0, 'date': Timestamp('2022-02-07 00:00:00')},
 {'weight': 60.9, 'date': Timestamp('2022-02-14 00:00:00')},
 {'weight': 60.0, 'date': Timestamp('2022-02-28 00:00:00')},
 {'weight': 60.6, 'date': Timestamp('2022-03-14 00:00:00')},
 {'weight': 60.8, 'date': Timestamp('2022-03-21 00:00:00')},
 {'weight': 61.9, 'date': Timestamp('2022-03-28 00:00:00')},
 {'weight': 62.1, 'date': Timestamp('2022-04-25 00:00:00')},
 {'weight': 62.2, 'date': Timestamp('2022-05-30 00:00:00')},
 {'weight': 63.5, 'date': Timestamp('2022-06-13 00:00:00')},
 {'weight': 64.6, 'date': Timestamp('2022-06-20 00:00:00')},
 {'weight': 64.0, 'date': Timestamp('2022-07-04 00:00:00')},
 {'weight': 64.2, 'date': Timestamp('2022-07-11 00:00:00')},
 {'weight': 65.0, 'date': Timestamp('2022-08-22 00:00:00')},
 {'weight': 64.0, 'date': Timestamp('2022-08-29 00:00:00')},
 {'weight': 63.1, 'date': Timestamp('2022-09-05 00:00:00')},
 {'weight': 61.8, 'date': Timestamp('2022-09-12 00:00:00')},
 {'weight': 61.6, 'date'

In [8]:
u = client.users.get_users()[0]
u["id"]

1

In [9]:
for weight in weights:
    value = weight["weight"]
    date = weight["date"].date()
    try:
        response = client.weights.create_weight(weight=value, date=str(date), user_id=u["id"])
        print(f"Created Weight {value}")
    except Exception as e:
        print(e)

Created Weight 60.0
Created Weight 60.9
Created Weight 60.0
Created Weight 60.6
Created Weight 60.8
Created Weight 61.9
Created Weight 62.1
Created Weight 62.2
Created Weight 63.5
Created Weight 64.6
Created Weight 64.0
Created Weight 64.2
Created Weight 65.0
Created Weight 64.0
Created Weight 63.1
Created Weight 61.8
Created Weight 61.6
Created Weight 60.7
Created Weight 58.7
Created Weight 58.8
Created Weight 57.9
Created Weight 56.7
Created Weight 58.2
Created Weight 59.8
Created Weight 61.8
Created Weight 61.0
Created Weight 61.4
Created Weight 61.4
Created Weight 61.4
Created Weight 61.3
Created Weight 62.0
Created Weight 62.4
Created Weight 61.7
Created Weight 62.6
Created Weight 61.9
Created Weight 62.5
Created Weight 63.2
Created Weight 63.2
Created Weight 62.1
Created Weight 63.6
Created Weight 63.3
Created Weight 65.8
Created Weight 66.8
Created Weight 67.1
Created Weight 67.7
Created Weight 67.9
Created Weight 69.5
Created Weight 69.7
Created Weight 70.5
Created Weight 71.6


# Muscle Groups

In [10]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="musclegroups")
df.head()

,name
0,chest
1,triceps
2,biceps
3,shoulders
4,quadriceps


In [11]:
for name in df.name.tolist():
    try:
        response = client.musclegroups.create_musclegroup(name=name)
        print(f"Created Muscle Group {name}")
    except Exception as e:
        print(e)

Created Muscle Group chest
Created Muscle Group triceps
Created Muscle Group biceps
Created Muscle Group shoulders
Created Muscle Group quadriceps
Created Muscle Group hamstrings
Created Muscle Group calves
Created Muscle Group core
Created Muscle Group back
Created Muscle Group glutes


# Exercises

In [12]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="exercises")
df.head()

,name,musclegroups
0,Bench Press,"chest, triceps"
1,Lean-In Cable Row,"back, biceps"
2,Dumbbell Shoulder Press Neutral Grip,shoulders
3,Lat Pulldown Neutral Grip,"back, biceps"
4,Rope Hammer Curls,biceps


In [13]:
mgs = pd.DataFrame(client.musclegroups.get_musclegroups())
mgs.head()

,name,id
0,chest,1
1,triceps,2
2,biceps,3
3,shoulders,4
4,quadriceps,5


In [14]:
for ix, exercise in df.iterrows():
    name = exercise["name"]
    mg_names = [mg.strip() for mg in exercise.musclegroups.split(",")]
    mg_ids = mgs[mgs["name"].isin(mg_names)].id.to_list()
    try:
        response = client.exercises.create_exercise(name=name, musclegroup_ids=mg_ids)
        print(f"Created Exercise {name}")
    except Exception as e:
        print(e)

Created Exercise Bench Press
Created Exercise Lean-In Cable Row
Created Exercise Dumbbell Shoulder Press Neutral Grip
Created Exercise Lat Pulldown Neutral Grip
Created Exercise Rope Hammer Curls
Created Exercise Cable Lateral Raises
Created Exercise Tricep Pushdown
Created Exercise V-Bar Pushdown
Created Exercise Barbell Back Squats
Created Exercise Barbell Stiff Leg Deadlift
Created Exercise Dumbbell Bulgarian Split Squats Single Arm
Created Exercise Hyperextensions
Created Exercise Rollouts
Created Exercise Military Press
Created Exercise Pull Ups Neutral Grip
Created Exercise Incline Dumbbell Bench Press Neutral Grip
Created Exercise Dumbbell Prone Rows
Created Exercise Seated Dumbbell Hammer Curls
Created Exercise Seated Dumbbell Lat Raises
Created Exercise Dumbbell Skull Crushers
Created Exercise Trap Bar Deadlift
Created Exercise Dumbbell Split Squats
Created Exercise Lying Leg Curls
Created Exercise Leg Press
Created Exercise Leg Raises
Created Exercise Hack Squat
Created Exerc

# Workout Routines

In [15]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="workoutroutines")
df = df.rename(columns={"name": "routine_name"})
df.head()

,routine_name,exercise,reps
0,Wave Loading 321 Lower 1,Trap Bar Deadlift,3
1,Wave Loading 321 Lower 1,Trap Bar Deadlift,2
2,Wave Loading 321 Lower 1,Trap Bar Deadlift,1
3,Wave Loading 321 Lower 1,Trap Bar Deadlift,3
4,Wave Loading 321 Lower 1,Trap Bar Deadlift,2


In [16]:
routines = df["routine_name"].unique()
routines

array(['Wave Loading 321 Lower 1', 'Wave Loading 321 Upper 1',
       'Wave Loading 321 Lower 2', 'Wave Loading 321 Upper 2',
       'Wave Loading 642 Lower 1', 'Wave Loading 642 Upper 1',
       'Wave Loading 642 Lower 2', 'Wave Loading 642 Upper 2',
       'Hypertrophy Legs', 'Hypertrophy Chest and Back',
       'Hypertrophy Shoulders and Arms', 'Arm Block Chest',
       'Arm Block Back', 'Arm Block Legs', 'Arm Block Arms',
       'Arm Block Legs 2', 'Upper Lower 2 Sets Upper 2',
       'Upper Lower 2 Sets Lower 2', 'Upper Lower 2 Sets Upper 1',
       'Upper Lower 2 Sets Lower 1', 'Upper Lower 3 Sets Lower 1',
       'Upper Lower 3 Sets Upper 1', 'Upper Lower 3 Sets Lower 2',
       'Upper Lower 3 Sets Upper 2'], dtype=object)

In [17]:
exs = pd.DataFrame(client.exercises.get_exercises())
exs.head()

,name,id
0,bench press,1
1,lean-in cable row,2
2,dumbbell shoulder press neutral grip,3
3,lat pulldown neutral grip,4
4,rope hammer curls,5


In [18]:
df["exercise_lower"] = df.exercise.str.lower()
df.head()

,routine_name,exercise,reps,exercise_lower
0,Wave Loading 321 Lower 1,Trap Bar Deadlift,3,trap bar deadlift
1,Wave Loading 321 Lower 1,Trap Bar Deadlift,2,trap bar deadlift
2,Wave Loading 321 Lower 1,Trap Bar Deadlift,1,trap bar deadlift
3,Wave Loading 321 Lower 1,Trap Bar Deadlift,3,trap bar deadlift
4,Wave Loading 321 Lower 1,Trap Bar Deadlift,2,trap bar deadlift


In [19]:
df_all = df.merge(right=exs, left_on="exercise_lower", right_on="name").set_index("routine_name")[["reps", "id"]]
df_all = df_all.reset_index().set_index(["routine_name", "id"])
df_all

reps
routine_name               id      
Wave Loading 321 Lower 1   21     3
                           21     2
                           21     1
                           21     3
                           21     2
...                             ...
Upper Lower 3 Sets Upper 2 39    20
                           39    20
                           37    12
                           37    12
                           37    20

[452 rows x 1 columns]

In [20]:
for routine in routines:
    all_exercises = []
    exercises = df_all.loc[routine].index.unique()
    for exercise in exercises:
        sets = df_all.loc[routine, exercise].to_dict(orient="records")
        data = {"id": exercise, "planned_sets": sets}
        all_exercises.append(data)
    try:
        response = client.workout_routines.create_workout_routine(name=routine, exercises=all_exercises, user_id=u["id"])
        print(f"Created Routine {routine}")
    except Exception as e:
        print(e)

C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(ori

Created Routine Wave Loading 321 Lower 1


C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(ori

Created Routine Wave Loading 321 Upper 1


C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(ori

...

Created Routine Upper Lower 3 Sets Upper 1


C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(ori

Created Routine Upper Lower 3 Sets Lower 2


C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\1573314857.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[routine, exercise].to_dict(ori

Created Routine Upper Lower 3 Sets Upper 2


# Workouts

In [21]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="workouts")
df["workoutroutine"] = df["workoutroutine"].str.lower()
df["exercise"] = df["exercise"].str.lower()
df.head()

,workoutroutine,date,exercise,weight,reps
0,arm block chest,2024-10-28,bench press,72.5,8
1,arm block chest,2024-10-28,bench press,72.5,5
2,arm block chest,2024-10-28,bench press,60.0,11
3,arm block chest,2024-10-28,dips,0.0,10
4,arm block chest,2024-10-28,dips,0.0,10


In [22]:
dates = df["date"].unique()
dates = [str(date.date()) for date in dates]

In [23]:
exs = pd.DataFrame(client.exercises.get_exercises())
exs = exs.rename(columns={"id": "exercise_id", "name": "exercise_name"})
exs.head()

,exercise_name,exercise_id
0,bench press,1
1,lean-in cable row,2
2,dumbbell shoulder press neutral grip,3
3,lat pulldown neutral grip,4
4,rope hammer curls,5


In [24]:
routines = pd.DataFrame(client.workout_routines.get_workout_routines())
routines = routines.rename(columns={"name": "routine_name", "id": "routine_id"})
routines

,routine_name,routine_id,routine_exercises,user_id
0,wave loading 321 lower 1,1,"[{'exercise': {'name': 'trap bar deadlift', 'i...",1
1,wave loading 321 upper 1,2,"[{'exercise': {'name': 'bench press', 'id': 1}...",1
2,wave loading 321 lower 2,3,"[{'exercise': {'name': 'barbell back squats', ...",1
3,wave loading 321 upper 2,4,"[{'exercise': {'name': 'military press', 'id':...",1
4,wave loading 642 lower 1,5,"[{'exercise': {'name': 'trap bar deadlift', 'i...",1
5,wave loading 642 upper 1,6,"[{'exercise': {'name': 'bench press', 'id': 1}...",1
6,wave loading 642 lower 2,7,"[{'exercise': {'name': 'barbell back squats', ...",1
7,wave loading 642 upper 2,8,"[{'exercise': {'name': 'military press', 'id':...",1
8,hypertrophy legs,9,"[{'exercise': {'name': 'lying leg curls', 'id'...",1
9,hypertrophy chest and back,10,[{'exercise': {'name': 'incline chest press ma...,1


In [25]:
df_all = df.merge(right=exs, left_on="exercise", right_on="exercise_name").merge(right=routines, left_on="workoutroutine", right_on="routine_name")[["date", "exercise_id", "routine_id", "weight", "reps"]]
dates = df_all.date.dt.date.astype(str).unique()
df_all = df_all.set_index(["date", "routine_id", "exercise_id"])
df_all

weight  reps
date       routine_id exercise_id              
2024-10-28 12         1              72.5     8
                      1              72.5     5
                      1              60.0    11
                      36              0.0    10
                      36              0.0    10
...                                   ...   ...
2024-12-01 12         8              20.0    20
                      8              20.0    18
                      8              20.0    16
                      39              0.0    16
                      39              0.0    15

[1030 rows x 2 columns]

In [26]:
for date in dates:
    routine_ids = df_all.loc[date].index.get_level_values(0).unique().values
    for routine_id in routine_ids:
        exercise_ids = df_all.loc[(date, routine_id)].index.unique().values
        all_exercises = []
        for exercise_id in exercise_ids:
            sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
            data = {"id": int(exercise_id), "sets": sets}
            all_exercises.append(data)
    try:
        response = client.workouts.create_workout(date=date, exercises=all_exercises, user_id=u["id"], workoutroutine_id=int(routine_id))
        print(f"Created workout for date {date}")
    except Exception as e:
        print(e)

C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  exercise_ids = df_all.loc[(date, routine_id)].index.unique().values
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  s

Created workout for date 2024-10-28


C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  exercise_ids = df_all.loc[(date, routine_id)].index.unique().values
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  s

Created workout for date 2024-10-29


C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  exercise_ids = df_all.loc[(date, routine_id)].index.unique().values
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  s

...

Created workout for date 2024-11-27


C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  exercise_ids = df_all.loc[(date, routine_id)].index.unique().values
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  s

Created workout for date 2024-11-30


C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  exercise_ids = df_all.loc[(date, routine_id)].index.unique().values
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine_id, exercise_id)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_8276\2859303871.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  s

Created workout for date 2024-12-01
